In [ ]:
from pyomo.environ import *
import pandas as pd

In [ ]:
# import data

gen = pd.read_excel('3bus_data_lecture.xlsx',sheet_name='gen',index_col='GeneratorName')
line = pd.read_excel('3bus_data_lecture.xlsx','line',index_col='lineID')
demand = pd.read_excel('3bus_data_lecture.xlsx','demand',index_col='DemandID')


# gen = pd.read_excel('5bus_data.xlsx',sheet_name='gen',index_col='GeneratorName')
# line = pd.read_excel('5bus_data.xlsx','line',index_col='lineID')
# demand = pd.read_excel('5bus_data.xlsx','demand',index_col='DemandID')

In [ ]:
gen

In [ ]:
# create sets, iterable objects
gens = gen.index #.tolist()
lines = line.index #.tolist()
demands = demand.index #.tolist()
buses = gen['bus'].unique()#.tolist()
# buses= ['A','B','C','D','E']

In [ ]:
# Create Model
m = ConcreteModel()

m.dual = Suffix(direction=Suffix.IMPORT)

In [ ]:
# Create Variables
m.power = Var(gens, domain=NonNegativeReals) # did not find a good syntax for bounds, have to use constraints. 
m.flow = Var(lines)
m.theta = Var(buses)


In [ ]:
# Create Objective Function
exp_obj = sum([m.power[g]*gen.loc[g,'Cost'] for g in gens])

In [ ]:
print(exp_obj)

In [ ]:
m.cost = Objective(expr=exp_obj, sense=minimize)

In [ ]:
# Create Bus Balance Constraints
m.bus_bal = ConstraintList()
for i in buses:
    expr = (sum([m.power[g] for g in gen[gen['bus']==i].index])
             -sum([demand['MW'][d] for d in demand[demand['bus']==i].index])
             -sum([m.flow[k] for k in line[line['from']==i].index])
             +sum([m.flow[k] for k in line[line['to']==i].index]))
    print(expr)
    m.bus_bal.add(expr==0)
    


In [ ]:
# Create Flow equation constraint
m.flow_eq = ConstraintList()
for k in lines:
    flow_eq = (m.flow[k] == (round(1/line['reactance'][k],2)*(m.theta[line['from'][k]]-m.theta[line['to'][k]])))
    print(flow_eq)
    m.flow_eq.add(flow_eq)

# m.flow_eq = Constraint(Any)
# for k in lines:
#     m.flow_eq[k] = (m.flow[k] == (round(1/line['reactance'][k],2)*(m.theta[line['from'][k]]-m.theta[line['to'][k]])))
#     print(m.flow_eq[k])


In [ ]:
# Create Flow limit constraint
m.flow_limit = ConstraintList()
for k in lines:
    m.flow_limit.add(inequality(-line['capacity'][k], m.flow[k], line['capacity'][k]))
    #m.flow_limit.add(-line['capacity'][k]<=m.flow[k]<=line['capacity'][k])

In [ ]:
# Designate a Reference bus
m.refBus = ConstraintList()
expr = (m.theta[1]==0)  
#expr = (m.theta['A']==0)
m.refBus.add(expr)


In [ ]:
# Create Generator Capacity Limit Constraint

In [ ]:
m.gen_cap = ConstraintList()
for g in gens:
    expr = m.power[g]<=gen['Capacity'][g]
    print(expr)
    m.gen_cap.add(expr)

In [ ]:
SolverFactory('glpk').solve(m)

In [ ]:
m.display()

In [ ]:
m.dual[m.bus_bal[2]]